In [ ]:
%load_ext autoreload

In [ ]:
import jax

jax.device_count(), jax.devices()[0].device_kind

In [ ]:
from whisper_stream import load_data_samples_from_path
from whisper_stream.pipelines import (
    JAXStreamablePipeline,
    JAXValidDtypesMapping,
    JAXScalarDType,
    JAXCheckpoints,
    JAXValidTasks,
)
from whisper_stream.logger import LOG_LEVEL_NAMES
import time

%autoreload 2

In [ ]:
checkpoint: JAXCheckpoints = "openai/whisper-tiny"
model_dtype: JAXScalarDType = JAXValidDtypesMapping["BFLOAT16"]
task: JAXValidTasks = "transcribe"
language: str = "english"
return_timestamps: bool = True
batch_size: int = 32
log_level: LOG_LEVEL_NAMES = "INFO"

run_opts = {"batch_size": batch_size, "return_timestamps": return_timestamps, "language": language, "task": task}

# construct
pipeline = JAXStreamablePipeline(
    checkpoint=checkpoint, dtype=model_dtype, batch_size=batch_size, min_log_level=log_level
)

In [ ]:
# Load data
pipeline_data: bytes = load_data_samples_from_path("audio_1.mp3", binary_mode=True)  # 4s
pipeline_data_large: bytes = load_data_samples_from_path("tryst.mp3", binary_mode=True)  # 4:44s

In [ ]:
# initialize & warmup
%time pipeline.initialize_pipeline(**run_opts, use_experimental_cache=True)

In [ ]:
# should be warmed up now (time should be similar to # small data)
%time list(pipeline(pipeline_data, **run_opts))

In [ ]:
# small data
%time list(pipeline(pipeline_data, **run_opts))

In [ ]:
# small data in batch
%time list(pipeline([pipeline_data] * 10, **run_opts))

In [ ]:
# chunkable data
%time list(pipeline(pipeline_data_large, **run_opts))

In [ ]:
# chunkable data in batches
%time list(pipeline([pipeline_data_large] * 32, **run_opts))

In [ ]:
mixed_mode_data: list[bytes] = [pipeline_data_large, pipeline_data, pipeline_data, pipeline_data] * 4

In [ ]:
# mixed data, received as it comes, using default `smallest` strategy the smaller files will come in larger batches first
start: float = time.time()
for data in pipeline(mixed_mode_data, strategy="smallest", **run_opts):
    print({"num_items": len(data)}, end="\n")
    print({"data": data, "time_taken": f"{time.time() - start:.2}s"}, end="\n")
    print("-" * 40, end="\n")
    start = time.time()